In [1]:
from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import TwoPointCrossOver

import torch

In [2]:
@vectorized
def fitness(x: torch.Tensor) -> torch.Tensor:
    return torch.linalg.norm(x, dim=-1)

In [3]:
problem = Problem(
    "min",
    fitness,
    solution_length=5,
    initial_bounds=(
        [-1, -2, -100, -1000, 10.0],
        [1, 2, 100, 1000, 10.5],
    ),
    dtype=torch.float32,
)

problem

[2023-03-04 03:35:15] INFO     < 3683> evotorch.core: Instance of `Problem` (id:139853880212112) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-03-04 03:35:15] INFO     < 3683> evotorch.core: Instance of `Problem` (id:139853880212112) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-03-04 03:35:15] INFO     < 3683> evotorch.core: Instance of `Problem` (id:139853880212112) -- The `device` of the problem is set as cpu
[2023-03-04 03:35:15] INFO     < 3683> evotorch.core: Instance of `Problem` (id:139853880212112) -- The number of actors that will be allocated for parallelized evaluation is 0


In [4]:
def my_own_gaussian_mutation(x: torch.Tensor) -> torch.Tensor:
    # The default GaussianMutation of EvoTorch does not (yet) support different standard deviation values
    # per variable. However, we can define our own mutation operator which adds noise of different magnitudes
    # to different variables, like in this example:
    [_, solution_length] = x.shape
    dtype = x.dtype
    device = x.device

    # Generate Gaussian noise where each column has its own magnitude
    noise = (
        torch.randn(solution_length, dtype=dtype, device=device)
        * torch.tensor([1, 2, 100, 1000, 0.1], dtype=dtype, device=device)
    )

    return x + noise

In [5]:
searcher = GeneticAlgorithm(
    problem,
    popsize=100,
    operators=[
        TwoPointCrossOver(problem, tournament_size=4),
        my_own_gaussian_mutation,
    ],
)
searcher

In [6]:
searcher.step()  # Take just one step. Just to see how the population looks like after one generation.

In [7]:
list(searcher.population[:10])

[<Solution values=tensor([ -0.9663,  -0.9670, -15.7090,  13.5809,  10.1975]), evals=tensor([23.1748])>,
 <Solution values=tensor([ -0.7827,   1.1606, -24.6908,  -4.2712,  10.0613]), evals=tensor([27.0382])>,
 <Solution values=tensor([-1.3241e-02, -4.9799e-01, -3.2429e+01,  1.3723e+00,  1.0246e+01]), evals=tensor([34.0410])>,
 <Solution values=tensor([ -0.2067,  -0.8951, -24.7820, -32.4819,  10.4979]), evals=tensor([42.1933])>,
 <Solution values=tensor([  0.4632,   1.6428, -35.7997,  25.6877,  10.2499]), evals=tensor([45.2709])>,
 <Solution values=tensor([  0.5060,  -1.1146, -23.6713,  37.5663,  10.3579]), evals=tensor([45.6107])>,
 <Solution values=tensor([ -0.8691,   1.7651, -59.3664,  12.5272,  10.3649]), evals=tensor([61.5841])>,
 <Solution values=tensor([ 0.7949,  1.3747,  5.4854, 70.8790, 10.1664]), evals=tensor([71.8318])>,
 <Solution values=tensor([-2.7432e-02, -1.9530e+00, -4.1382e+01, -6.3109e+01,  1.0044e+01]), evals=tensor([76.1573])>,
 <Solution values=tensor([ -0.6017,  -1